# Simulate QSO spectra.

The purpose of this notebook is to simulate a bunch of QSO spectra using [simqso](https://github.com/moustakas/simqso) and the default (PCA-based) QSO template-generating code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

In [2]:
from desisim.templates import SIMQSO, QSO

In [3]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2

In [4]:
plt.style.use('seaborn-talk')
%matplotlib inline

### Specify the random seed and the magnitude and redshift priors.

In [5]:
seed = 1
rand = np.random.RandomState(seed)

In [6]:
nmodel = 100
mag = rand.uniform(18, 20, nmodel) # r-band
redshift = rand.uniform(2, 4, nmodel)

In [7]:
minwave, maxwave = 2500, 10e4

#### Generate spectra using both SIMQSO() and QSO().

In [8]:
simqso = SIMQSO(minwave=minwave, maxwave=maxwave, cdelt=2.0)

ToDo: Update to LegacySurvey filters!! What is 1450?


In [9]:
%time flux, wave, meta = simqso.make_templates(nmodel, seed=seed, redshift=redshift, mag=mag, nocolorcuts=True)

> /Users/ioannis/repos/desihub/desisim/py/desisim/templates.py(2221)make_templates()
-> qsos = QsoSimPoints([mag, redshift], cosmo=self.cosmo, units='flux')
(Pdb) mag
array([ 18.83404401,  19.44064899,  18.00022875,  18.60466515,
        18.29351178,  18.18467719,  18.37252042,  18.69112145,
        18.79353495,  19.07763347,  18.83838903,  19.370439  ,
        18.4089045 ,  19.75623487,  18.05477519,  19.34093502,
        18.8346096 ,  19.11737966,  18.28077388,  18.39620298,
        19.60148914,  19.93652315,  18.62684836,  19.38464523,
        19.7527783 ,  19.78921333,  18.17008842,  18.07810957,
        18.33966084,  19.75628501,  18.19669367,  18.84221525,
        19.91577906,  19.06633057,  19.38375423,  18.63103126,
        19.37300186,  19.66925134,  18.03657655,  19.50028863,
        19.97772218,  19.49633131,  18.56088798,  19.57855866,
        18.20645201,  18.89578705,  19.81719101,  18.5872283 ,
        18.57555068,  18.26005714,  18.03873392,  19.35767107,
        18.423

BdbQuit: 

In [10]:
stop

NameError: name 'stop' is not defined

In [ ]:
meta

In [ ]:
qso = QSO(minwave=minwave, maxwave=maxwave, cdelt=2.0)

In [ ]:
%time qflux, _, qmeta = qso.make_templates(nmodel, seed=seed, redshift=redshift, mag=mag, nocolorcuts=True)

In [ ]:
def compare_templates(nplot=12, ncol=4, xlim=None):
    """Plot a random sampling of the basis templates."""
    
    if xlim is None:
        xlim = (2500, 6000)
    
    nspec, npix = flux.shape
    nrow = np.ceil(nplot / ncol).astype('int')
    these = rand.choice(nspec, nplot, replace=False)
    these = np.sort(these)

    fig, ax = plt.subplots(nrow, ncol, figsize=(3*ncol, 2.5*nrow), sharey=False, sharex=True)
    for ii, (thisax, indx) in enumerate(zip(ax.flat, these)):
        thisax.plot(wave, flux[indx, :], label='SIMQSO')
        thisax.plot(wave, qflux[indx, :], alpha=0.7, label='QSO')
        thisax.xaxis.set_major_locator(plt.MaxNLocator(3))
        thisax.set_xlim(xlim)
        ww = (wave > xlim[0]) * (wave < xlim[1])
        ylim = (flux[indx, ww].min(), flux[indx, ww].max())
        thisax.set_ylim(ylim)
        thisax.yaxis.set_ticklabels([])
        if xlim[1] > 1e4:
            thisax.set_xscale('log')
            thisax.set_yscale('log')
        if ii == 0:
            thisax.legend(loc='upper left')
    fig.subplots_adjust(wspace=0.05, hspace=0.05)

In [ ]:
compare_templates()

In [ ]:
compare_templates(xlim=(6000, maxwave))

### Compare the two template sets in color-color and color-redshift space.

In [ ]:
def flux2colors(cat):
    """Convert DECam/WISE fluxes to magnitudes and colors."""
    colors = dict()
    #with warnings.catch_warnings(): # ignore missing fluxes (e.g., for QSOs)
    #    warnings.simplefilter('ignore')
    colors['g'] = 22.5 - 2.5 * np.log10(cat['FLUX_G'])
    colors['r'] = 22.5 - 2.5 * np.log10(cat['FLUX_R'])
    colors['z'] = 22.5 - 2.5 * np.log10(cat['FLUX_Z'])
    colors['gr'] = colors['g'] - colors['r']
    colors['gz'] = colors['g'] - colors['z']
    colors['rz'] = colors['r'] - colors['z']
    colors['grz'] = 22.5-2.5*np.log10(cat['FLUX_G'] + 0.8 * cat['FLUX_R'] +  0.5 * cat['FLUX_G'] / 2.3)

    with np.errstate(invalid='ignore'):
        colors['W'] = 22.5-2.5*np.log10(0.75 * cat['FLUX_W1'] + 0.25 * cat['FLUX_W2'])
        colors['rW'] = colors['r'] - colors['W']
    
    return colors

In [ ]:
colors = flux2colors(meta)
qcolors = flux2colors(qmeta)

In [ ]:
def qso_colorbox(ax, plottype='grz'):
    """Draw the QSO selection boxes."""
    rmaglim = 22.7
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    if plottype == 'grz-r':
        verts = [(xlim[0]-0.05, 17.0),
                 (22.7, 17.0),
                 (22.7, ylim[1]+0.05),
                 (xlim[0]-0.05, ylim[1]+0.05)
                ]
    if plottype == 'rW1-rz':
        verts = None
        ax.axvline(x=-0.3, ls='--', color='k')
        ax.axvline(x=1.3, ls='--', color='k')

    if plottype == 'gr-rz':
        verts = [(-0.3, 1.3),
                 (1.1, 1.3),
                 (1.1, ylim[0]-0.05),
                 (-0.3, ylim[0]-0.05)
                ]
    if verts:
        ax.add_patch(Polygon(verts, fill=False, ls='--', color='k'))

In [ ]:
def qa_colorcolor():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.scatter(colors['rz'], colors['gr'], s=14, label='SIMQSO')
    ax1.scatter(qcolors['rz'], qcolors['gr'], s=14, label='QSO')
    ax1.set_xlabel('$r - z$')
    ax1.set_ylabel('$g - r$')
    ax1.set_xlim(-1, 2.2)
    ax1.set_ylim(-1, 2.0)
    ax1.legend(loc='upper right')
    qso_colorbox(ax1, 'gr-rz')
    
    ax2.scatter(colors['gz'], colors['rW'], s=14, label='SIMQSO')
    ax2.set_xlabel('$g - z$')
    ax2.set_ylabel('$r - W$')
    ax2.set_xlim(-0.3, 2)
    ax2.set_ylim(-1, 3)
    ax2.legend(loc='upper right')
    gzaxis = np.linspace(-0.5, 2.0, 50)
    ax2.plot(gzaxis, np.polyval([1.0, -1.0], gzaxis))
    #qso_colorbox(ax1, 'gr-rz')
    plt.subplots_adjust(wspace=0.3)

In [ ]:
qa_colorcolor()